In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("all_vals_race1_interestrate1.csv")



C:\Users\bencu\AppData\Local\Temp\ipykernel_37408\785749740.py:4: DtypeWarning: Columns (101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_vals_race1_interestrate1.csv")


In [8]:
df.rename(
    columns={
        "occupancy_type_dum_1": "occupancy_primary",
        "occupancy_type_dum_2": "occupancy_secondary",
    },
    inplace=True,
)

df["occupancy_unknown"] = (
    (df["occupancy_primary"] == 0) & (df["occupancy_secondary"] == 0)
).astype(int)

df.rename(
    columns={
        "investor_type_dum_1": "investor_fannie_mae",
        "investor_type_dum_2": "investor_ginnie_mae",
        "investor_type_dum_3": "investor_freddie_mac",
    },
    inplace=True,
)

df["investor_gse"] = (
    (df["investor_fannie_mae"] == 1)
    | (df["investor_ginnie_mae"] == 1)
    | (df["investor_freddie_mac"] == 1)
).astype(int)

df["investor_other"] = (
    (df["investor_fannie_mae"] == 0)
    & (df["investor_ginnie_mae"] == 0)
    & (df["investor_freddie_mac"] == 0)
).astype(int)

df.drop(
    columns=[
        "investor_fannie_mae",
        "investor_ginnie_mae",
        "investor_freddie_mac",
    ],
    inplace=True,
    axis=1,
)

In [14]:
columns = df.columns

for c in columns:
    print(c)

IsTestData
IsCalibrateData
applicant_income
income_miss
ltv_ratio_fill
fico_orig_miss
fico_orig_fill
orig_amt
cur_int_rate
sato
log_org_amt
loan_Conventional
loan_FHA
loan_VA
occupancy_primary
occupancy_secondary
jumbo_flg_dum_n
loan_purpose_dum_1
loan_purpose_dum_2
coapplicant_dum_0
term_nmon_dum_120
term_nmon_dum_180
term_nmon_dum_240
loan_type_mcdash_dum_c
prepayment
orig_year_dum_2018
orig_year_dum_2019
orig_year_dum_2020
orig_year_dum_2021
orig_year_dum_2022
orig_year_dum_2023
orig_year_dum_2024
prop_state_dum_AK
prop_state_dum_AL
prop_state_dum_AR
prop_state_dum_AZ
prop_state_dum_CA
prop_state_dum_CO
prop_state_dum_CT
prop_state_dum_DC
prop_state_dum_DE
prop_state_dum_FL
prop_state_dum_GA
prop_state_dum_HI
prop_state_dum_IA
prop_state_dum_ID
prop_state_dum_IL
prop_state_dum_IN
prop_state_dum_KS
prop_state_dum_KY
prop_state_dum_LA
prop_state_dum_MA
prop_state_dum_MD
prop_state_dum_ME
prop_state_dum_MI
prop_state_dum_MN
prop_state_dum_MO
prop_state_dum_MS
prop_state_dum_MT
prop_sta

In [18]:
continuous_cols = [
    "applicant_income",
    "ltv_ratio_fill",
    "fico_orig_fill",
    "orig_amt",
    "cur_int_rate",
]

missing_continuous = [col for col in continuous_cols if col not in df.columns]
if missing_continuous:
    raise ValueError(f"Missing expected continuous columns: {missing_continuous}")


def summarize_continuous(data, cols):
    summary_rows = []
    for col in cols:
        series = data[col].astype(float)
        summary_rows.append(
            {
                "column": col,
                "count": int(series.count()),
                "missing": int(series.isna().sum()),
                "missing_pct": round(series.isna().mean() * 100, 2),
                "mean": series.mean(),
                "std": series.std(ddof=1),
                "min": series.min(),
                "p1": series.quantile(0.01),
                "q1": series.quantile(0.25),
                "median": series.quantile(0.5),
                "q3": series.quantile(0.75),
                "p99": series.quantile(0.99),
                "max": series.max(),
            }
        )
    return (
        pd.DataFrame(summary_rows)
        .set_index("column")
        .round({
            "mean": 2,
            "std": 2,
            "min": 2,
            "p1": 2,
            "q1": 2,
            "median": 2,
            "q3": 2,
            "p99": 2,
            "max": 2,
        })
    )

continuous_summary = summarize_continuous(df, continuous_cols)
print(continuous_summary)

                    count  missing  missing_pct       mean        std  \
column                                                                  
applicant_income  1297683        0          0.0     178.54    2016.78   
ltv_ratio_fill    1297683        0          0.0       0.88       0.17   
fico_orig_fill    1297683        0          0.0     604.50     271.59   
orig_amt          1297683        0          0.0  435289.06  469553.27   
cur_int_rate      1297683        0          0.0       0.04       0.01   

                      min        p1         q1     median         q3  \
column                                                                 
applicant_income -3390.00      0.00      62.00      94.00     161.00   
ltv_ratio_fill       0.00      0.16       0.80       0.96       0.98   
fico_orig_fill       0.00      0.00     636.00     702.00     771.00   
orig_amt          5000.00  35000.00  195000.00  295000.00  505000.00   
cur_int_rate         0.01      0.02       0.03       0.0

In [20]:
def summarize_dummy_block(data, columns, group_name, labels=None, include_other=False, other_label="Other"):
    labels = labels or {}
    missing = [col for col in columns if col not in data.columns]
    if missing:
        raise ValueError(f"Missing expected categorical columns for {group_name}: {missing}")

    block = data[columns]
    total = len(block)
    counts = block.sum().astype(int)

    summary = pd.DataFrame(
        {
            "group": group_name,
            "category": [labels.get(col, col) for col in columns],
            "count": counts.values,
        }
    )
    summary["share"] = (summary["count"] / total).round(4)

    if include_other:
        covered_rows = int((block.sum(axis=1) > 0).sum())
        other_count = total - covered_rows
        if other_count > 0:
            summary = pd.concat(
                [
                    summary,
                    pd.DataFrame(
                        {
                            "group": [group_name],
                            "category": [other_label],
                            "count": [other_count],
                            "share": [round(other_count / total, 4)],
                        }
                    ),
                ],
                ignore_index=True,
            )

    return summary.sort_values(by="share", ascending=False).reset_index(drop=True)


categorical_configs = [
    {
        "group": "loan_type",
        "columns": ["loan_Conventional", "loan_FHA", "loan_VA"],
        "labels": {
            "loan_Conventional": "Conventional",
            "loan_FHA": "FHA",
            "loan_VA": "VA",
        },
        "include_other": True,
        "other_label": "Other / missing loan type",
    },
    {
        "group": "occupancy_type",
        "columns": ["occupancy_primary", "occupancy_secondary", "occupancy_unknown"],
        "labels": {
            "occupancy_primary": "Primary",
            "occupancy_secondary": "Secondary",
            "occupancy_unknown": "Unknown",
        },
    },
    {
        "group": "jumbo_flag",
        "columns": ["jumbo_flg_dum_n"],
        "labels": {"jumbo_flg_dum_n": "Non-jumbo"},
        "include_other": True,
        "other_label": "Jumbo",
    },
    {
        "group": "loan_purpose",
        "columns": ["loan_purpose_dum_1", "loan_purpose_dum_2"],
        "labels": {
            "loan_purpose_dum_1": "Purpose #1",
            "loan_purpose_dum_2": "Purpose #2",
        },
        "include_other": True,
        "other_label": "Other / missing purpose",
    },
    {
        "group": "term_length",
        "columns": ["term_nmon_dum_120", "term_nmon_dum_180", "term_nmon_dum_240"],
        "labels": {
            "term_nmon_dum_120": "120 months",
            "term_nmon_dum_180": "180 months",
            "term_nmon_dum_240": "240 months",
        },
        "include_other": True,
        "other_label": "Other term",
    },
]

categorical_summaries = []
for cfg in categorical_configs:
    categorical_summaries.append(
        summarize_dummy_block(
            df,
            columns=cfg["columns"],
            group_name=cfg["group"],
            labels=cfg.get("labels"),
            include_other=cfg.get("include_other", False),
            other_label=cfg.get("other_label", "Other"),
        )
    )

categorical_summary = pd.concat(categorical_summaries, ignore_index=True)
print(categorical_summary)

             group      category    count   share
0        loan_type           FHA   464733  0.3581
1        loan_type  Conventional   435686  0.3357
2        loan_type            VA   397264  0.3061
3   occupancy_type       Primary  1251727  0.9646
4   occupancy_type     Secondary    45679  0.0352
5   occupancy_type       Unknown      277  0.0002
6       jumbo_flag         Jumbo  1083056  0.8346
7       jumbo_flag     Non-jumbo   214627  0.1654
8     loan_purpose    Purpose #1  1297683  1.0000
9     loan_purpose    Purpose #2        0  0.0000
10     term_length    Other term  1262883  0.9732
11     term_length    180 months    24408  0.0188
12     term_length    240 months     6097  0.0047
13     term_length    120 months     4295  0.0033


In [22]:
print(df['default'].value_counts())
print(df["prepayment"].value_counts())

default
0    1214880
1      82803
Name: count, dtype: int64
prepayment
0    961019
1    336664
Name: count, dtype: int64
